In [1]:
import Pkg

In [2]:
Pkg.activate(".")

Activating environment at `~/DistancePlan/Project.toml`


In [3]:
using Revise

In [4]:
using StaticArrays

In [5]:
using Rotations

In [6]:
includet("src/DistancePlan.jl")

In [7]:
s1 = DistancePlan.Sphere([1. 1.], 2.)

Main.DistancePlan.Sphere([1.0 1.0], 2.0)

In [8]:
s2 = DistancePlan.Sphere([3. 1.], 1.)

Main.DistancePlan.Sphere([3.0 1.0], 1.0)

In [9]:
d = DistancePlan.norm(s2.center - s1.center)

2.0

In [10]:
x = 2 * s1.radius ^ 2 - s2.radius ^ 2 / (2 * s1.radius)

7.75

In [11]:
function cone_angle(sphere_a, sphere_b)
    assert(isapprox(sum((sphere_a.center - sphere_b.center) ^ 2), sphere_a.radius ^2))
    # "Align" the spheres along the x axis, and take an arbitrary orthogonal y axis.
    # Then we would have the following equations:
    #   x^2 + y^2 = R^2
    #   (x - R)^2 + y^2 = r^2
    # Solving for y^2:
    #   x^2 + y^2 = R^2         =>   y^2 = R^2 - x^2
    #   (x - R)^2 + y^2 = r^2   =>   y^2 = r^2 - (x - R)^2
    # Combining the equations:
    #   R^2 - x^2 = r^2 - (x - R)^2
    #   R^2 - x^2 - r^2 + (x^2 - 2xR + R^2) = 0
    #   -x^2 + x^2 - 2Rx + 2R^2 - r^2 = 0
    #   -2Rx = -2R^2 + r^2
    #   x = (2R^2 - r^2) / 2R
    x = (2 * sphere_a.radius ^ 2 - sphere_b.radius^2) / (2 * sphere_a.radius)
    # Now we need the angle, so we use cos(x / R):
    return cos(x / sphere_a.radius)
end

cone_angle (generic function with 1 method)

In [12]:
function lens_center(sphere_a, sphere_b)
    offset = sphere_a.center - sphere_b.center
    d = sqrt(sum(offset .^ 2))
    # "Align" the spheres along the x axis, and take an arbitrary orthogonal y axis.
    # Then we would have the following equations:
    #   x^2 + y^2 = R^2
    #   (x - d)^2 + y^2 = r^2
    # Solving for y^2:
    #   x^2 + y^2 = R^2         =>   y^2 = R^2 - x^2
    #   (x - d)^2 + y^2 = r^2   =>   y^2 = r^2 - (x - d)^2
    # Combining the equations:
    #   R^2 - x^2 = r^2 - (x - R)^2
    #   R^2 - x^2 - r^2 + (x^2 - 2xd + d^2) = 0
    #   -x^2 + x^2 - 2dx + d^2 + R^2 - r^2 = 0
    #   -2dx = -2R^2 + r^2
    #   x = (d^2 + R^2 - r^2) / 2d
    x = (d ^ 2 + sphere_a.radius ^ 2 - sphere_b.radius^2) / (2 * d)
    return sphere_a.center .+ offset * (x / d)
end

lens_center (generic function with 1 method)

In [139]:
function shared_sphere(sphere_a, sphere_b)
    offset = sphere_a.center - sphere_b.center
    d = sqrt(sum(offset .^ 2))
    if d == 0.
        if sphere_a.radius >= sphere_b.radius
            return sphere_b
        else
            return sphere_a
        end
    end
    # "Align" the spheres along the x axis, and take an arbitrary orthogonal y axis.
    # Then we would have the following equations:
    #   x^2 + y^2 = R^2
    #   (x - d)^2 + y^2 = r^2
    # Solving for y^2:
    #   x^2 + y^2 = R^2         =>   y^2 = R^2 - x^2
    #   (x - d)^2 + y^2 = r^2   =>   y^2 = r^2 - (x - d)^2
    # Combining the equations:
    #   R^2 - x^2 = r^2 - (x - R)^2
    #   R^2 - x^2 - r^2 + (x^2 - 2xd + d^2) = 0
    #   -x^2 + x^2 - 2dx + d^2 + R^2 - r^2 = 0
    #   -2dx = -2R^2 + r^2
    #   x = (d^2 + R^2 - r^2) / 2d
    x = (d ^ 2 + sphere_a.radius ^ 2 - sphere_b.radius^2) / (2 * d)
    point = sphere_a.center .+ offset * (x / d)
    y = sqrt(abs(sphere_a.radius ^ 2 - x ^ 2))
    result_sphere = DistancePlan.Sphere(point, y)
    return result_sphere
end

shared_sphere (generic function with 1 method)

In [14]:
using LinearAlgebra

In [45]:
struct Arm
    lengths
    joint_axes
#     joint_lows
#     joint_highs
end

ErrorException: invalid redefinition of constant Arm

In [16]:
Vector(s1.center[1]:.2:s2.center[1])

11-element Array{Float64,1}:
 1.0
 1.2
 1.4
 1.6
 1.8
 2.0
 2.2
 2.4
 2.6
 2.8
 3.0

In [17]:
function query_spheres(joint_locations, obstacles, points_per_link=5)
    Channel() do channel
        for i in 1:(length(joint_locations) - 1)
            start = joint_locations[i]
            stop = joint_locations[i + 1]
            for point in LinRange(start, stop, points_per_link)
                d = DistancePlan.distance(point, obstacles)
                put!(channel, (i, DistancePlan.Sphere(point, d)))
            end
        end
    end
end

query_spheres (generic function with 2 methods)

In [18]:
for (link_number, sphere) in query_spheres([[0., 0.], [1., 1.], [2., 1.]], [[.5, .6]])
    println(link_number, ": ", sphere)
end

1: Main.DistancePlan.Sphere([0.0, 0.0], 0.7810249675906654)
1: Main.DistancePlan.Sphere([0.25, 0.25], 0.4301162633521313)
1: Main.DistancePlan.Sphere([0.5, 0.5], 0.09999999999999998)
1: Main.DistancePlan.Sphere([0.75, 0.75], 0.29154759474226505)
1: Main.DistancePlan.Sphere([1.0, 1.0], 0.6403124237432849)
2: Main.DistancePlan.Sphere([1.0, 1.0], 0.6403124237432849)
2: Main.DistancePlan.Sphere([1.25, 1.0], 0.85)
2: Main.DistancePlan.Sphere([1.5, 1.0], 1.077032961426901)
2: Main.DistancePlan.Sphere([1.75, 1.0], 1.3124404748406688)
2: Main.DistancePlan.Sphere([2.0, 1.0], 1.5524174696260025)


In [19]:
LinRange(s1.center, s2.center, 3)

3-element LinRange{Array{Float64,2}}:
 [1.0 1.0],[2.0 1.0],[3.0 1.0]

Generally, the kinematic volume planning algorithm extension step works as follows:

For each linkage, sample distances along that linkage, producing a sequence of hyper-spheres.
For each pair of adjacent hyper-spheres in the sequence, compute the largest hyper-sphere which overlaps both of them, by finding the hyper-sphere one dimension lower. These overlap hyper-spheres are called the "safe spheres."
For each joint, compute the angle that joint can move in each safe sphere after it in the kinematic chain. Take the smallest of these angles for that joint. That is the joints "safe angle."

Save all joints patch angles (as well as the central angle) as a patch into the patch tree.

When executing motion within the patch, the sum of angles divided by their respective safe angles should be less than one. This produces a diagonal patch in the angle space.

The reason this works is that change in angle corresponds to change in the distance of the linkage to the safe circle. Specifically, it corresponds to a portion of the distance equal the ratio of the angle over the safe angle. Alternatively, this can be seen as adding the fractions of the arcs together which the different joints would sweep out. Since the arcs aren't overlapping (assuming linkages have non-zero length), the length of their arc lengths in sequence will always be greater than the distance between their joined endpoints (i.e. the triangle inequality). However, this bound is not tight. For example, linkages at orthogonal angles can get arbitrarily close to their full individual ranges of motion. Unfortunately, performing projections on a shape that tightly bounds the safe angle space corresponding to the safe spheres would be quite difficult, since every such shape would be different. This bound can be arbitrarily close to tight, if the two joints are close to each other, and the second linkage is very long.

In [20]:
struct PatchTree
  centers::Vector
  patch_sizes::Vector
  children::Vector{Vector{Int}}
  parent::Vector{Int}
  PatchTree() = new([], [], [], [])
end

In [170]:
function forward_kinematics(arm::Arm, origin, joint_angles)::Vector
    points = [origin]
    current_rotation = one(AngleAxis)
    for (joint_len, joint_axis, angle) in zip(arm.lengths, arm.joint_axes, joint_angles)
        joint_rotation = AngleAxis(angle, joint_axis[1], joint_axis[2], joint_axis[3])
        current_rotation = current_rotation * joint_rotation
#         current_rotation = joint_rotation * current_rotation
        linkage = SVector(joint_len, 0., 0.)
        rotated_linkage = current_rotation * linkage
        point = points[length(points)] .+ rotated_linkage
        push!(points, point)
    end
    return points
end

forward_kinematics (generic function with 1 method)

In [234]:
arm = Arm([2., 2.], [SVector(0., 0., 1.), SVector(0., 1., 0.)], [0., 0.], [2π, 2π])

Arm([2.0, 2.0], SArray{Tuple{3},Float64,1,3}[[0.0, 0.0, 1.0], [0.0, 1.0, 0.0]], [0.0, 0.0], [6.283185307179586, 6.283185307179586])

In [23]:
forward_kinematics(arm, SVector(0., 1., 0.), [π/2, -π/2])

[2.220446049250313e-16 -1.0 0.0; 1.0 2.220446049250313e-16 0.0; 0.0 0.0 1.0]
[1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]


3-element Array{SArray{Tuple{3},Float64,1,3},1}:
 [0.0, 1.0, 0.0]                  
 [4.440892098500626e-16, 3.0, 0.0]
 [2.0000000000000004, 3.0, 0.0]   

In [24]:
forward_kinematics(arm, SVector(0., 1., 0.), [0., 0.])

[1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]
[1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]


3-element Array{SArray{Tuple{3},Float64,1,3},1}:
 [0.0, 1.0, 0.0]
 [2.0, 1.0, 0.0]
 [4.0, 1.0, 0.0]

In [83]:
using MeshCat
vis = Visualizer()
open(vis)

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8701
└ @ MeshCat /home/kr/.julia/packages/MeshCat/GMobF/src/servers.jl:24


Process(`xdg-open http://127.0.0.1:8701`, ProcessExited(0))

Error handling websocket connection:
TaskFailedException:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")
Stacktrace:
 [1] handle_control_frame(::WebSockets.WebSocket{Sockets.TCPSocket}, ::WebSockets.WebSocketFragment) at /home/kr/.julia/packages/WebSockets/pc4iW/src/WebSockets.jl:376
 [2] read(::WebSockets.WebSocket{Sockets.TCPSocket}) at /home/kr/.julia/packages/WebSockets/pc4iW/src/WebSockets.jl:452
 [3] macro expansion at /home/kr/.julia/packages/WebIO/2mZPb/src/providers/mux.jl:38 [inlined]
 [4] (::WebIO.var"#109#110"{WebSockets.WebSocket{Sockets.TCPSocket},WebIO.WebSockConnection})() at ./task.jl:333
Stacktrace:
 [1] wait at ./task.jl:251 [inlined]
 [2] create_socket(::Dict{Any,Any}) at /home/kr/.julia/packages/WebIO/2mZPb/src/providers/mux.jl:44
 [3] (::Mux.var"#5#6"{Mux.var"#28#29"{Array{SubString{String},1}},typeof(WebIO.create_socket)})(::Function, ::Dict{Any,Any}) at /home/kr/.julia/packages/Mux/FeATY/src/Mux.jl:17
 [4] #1 at /home/kr/.ju

In [26]:
using WebIO, IJulia
WebIO.install_jupyter_labextension()

┌ Info: Using default Jupyter executable at `/home/kr/.virtualenvs/julia/bin/jupyter`; to use a different executable, see the documentation by running `?WebIO.install_jupyter_labextension`.
└ @ WebIO /home/kr/.julia/packages/WebIO/2mZPb/deps/jupyter.jl:141
┌ Error: error handling request
│   exception = (Base.IOError("stream is closed or unusable", 0), Base.StackTraces.StackFrame[check_open at stream.jl:328 [inlined], uv_write_async(::Sockets.TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:961, uv_write(::Sockets.TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:924, unsafe_write(::Sockets.TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:1007, unsafe_write at ConnectionPool.jl:134 [inlined], macro expansion at gcutils.jl:91 [inlined], write at io.jl:186 [inlined], closebody at Streams.jl:111 [inlined], closewrite(::HTTP.Streams.Stream{HTTP.Messages.Request,HTTP.ConnectionPool.Transaction{Sockets.TCPSocket}}) at Streams.jl:126, (::HTTP.Servers.var"#13#14"{WebSockets.var"#_servercoroutine#1

An error occured.
ValueError: No linked package for @webio/webio
See the log file for details:  /tmp/jupyterlab-debug-0zxj5kw_.log


Uninstalling @webio/jupyter-lab-provider from /home/kr/.virtualenvs/julia/share/jupyter/lab/extensions


[LabBuildApp] JupyterLab 1.2.1
[LabBuildApp] Building in /home/kr/.virtualenvs/julia/share/jupyter/lab


-

[LabBuildApp] Building jupyterlab assets (build:prod:minimize)




Process(`/home/kr/.virtualenvs/julia/bin/jupyter lab build`, ProcessExited(0))

In [27]:
using GeometryTypes
using Colors

In [28]:
delete!(vis)
blue_material = MeshPhongMaterial(color=RGB(0, 0, 1))
green_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.1))
red_material = MeshPhongMaterial(color=RGB(1, 0, 0))
joint_points = forward_kinematics(arm, SVector(0., 0., 1.), [π/4, -π/3])
for (i, v) in enumerate(joint_points)
    setobject!(vis["arm"][string(i)], HyperSphere(Point(v[1], v[2], v[3]), .1), blue_material)
end
for i in 1:(length(joint_points) - 1)
    start = joint_points[i]
    stop = joint_points[i + 1]
    for (j, point) in enumerate(LinRange(start, stop, 20))
        setobject!(vis["arm_line"][string(i)][string(j)], HyperSphere(Point(point), 0.01), blue_material)
    end
end
obstacles = [[.5, .6, .7], [3., 1., 2.], [2., 1., 2.]]
for (i, (link, sphere)) in enumerate(query_spheres(joint_points, obstacles))
    setobject!(vis["samples"][string(i)], HyperSphere(Point(sphere.center), sphere.radius), green_material)
end
for (i, obs) in enumerate(obstacles)
    setobject!(vis["obstacles"][string(i)], HyperSphere(Point{3}(obs), 0.05), red_material)
end

[0.7071067811865475 -0.7071067811865476 0.0; 0.7071067811865476 0.7071067811865475 0.0; 0.0 0.0 1.0]
[0.9659258262890685 0.25881904510252063 0.0; -0.25881904510252063 0.9659258262890685 -0.0; -0.0 0.0 1.0000000000000002]


In [156]:
function nearest_patch(tree::PatchTree, point)::Int
    return argmin([patch_distance(center, patch_size, point)
                   for (center, patch_size) in zip(tree.centers, tree.patch_sizes)])
end

nearest_patch (generic function with 1 method)

In [157]:
function relu(x)
    if x >= 0.
        return x
    else
        return 0.
    end
end

relu (generic function with 1 method)

In [158]:
function patch_distance(center, patch_size, point)
    return sum(relu.(abs.(center .- point) .- patch_size))
end

patch_distance (generic function with 2 methods)

In [192]:
function project(tree::PatchTree, patch::Int, point)
    center = tree.centers[patch]
    patch_size = tree.patch_sizes[patch]
    rel_point = abs.((point .- center) ./ patch_size)
    max_coord = maximum(rel_point)
#     println("max_coord: ", max_coord)
    signs = (1e-6 .+ (point .- center)) ./ (1e-6 .+ abs.(point .- center))
#     println("signs: ", signs)
#     println("center: ", center)
    offset = (signs .* abs.(rel_point .- max_coord))
#     println("offset: ", offset)
    projection = offset .+ center
#     println("projection: ", projection)
    return projection
end

project (generic function with 1 method)

In [210]:
function draw_joint_points(joint_points; prefix="arm")
    blue_material = MeshPhongMaterial(color=RGB(0, 0, 1))
    for (i, v) in enumerate(joint_points)
        setobject!(vis[prefix][string(i)], HyperSphere(Point(v[1], v[2], v[3]), .1), blue_material)
    end
    for i in 1:(length(joint_points) - 1)
        start = joint_points[i]
        stop = joint_points[i + 1]
        for (j, point) in enumerate(LinRange(start, stop, 20))
            setobject!(vis[prefix * "-line"][string(i)][string(j)], HyperSphere(Point{3}(point), 0.01), blue_material)
        end
    end
end

function draw_obstacles(obstacles)
    red_material = MeshPhongMaterial(color=RGB(1, 0, 0))
    for (i, obs) in enumerate(obstacles)
        setobject!(vis["obstacles"][string(i)], HyperSphere(Point{3}(obs), 0.05), red_material)
    end
end

function draw_free_spheres(free_spheres)
    green_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.1))
    for (i, (link, sphere)) in enumerate(free_spheres)
        setobject!(vis["samples"][string(i)], HyperSphere(Point{3}(sphere.center), sphere.radius), green_material)
    end
end

draw_free_spheres (generic function with 1 method)

In [198]:
function configuration_distance(arm::Arm, origin, joint_angles, obstacles)
#     draw_obstacles(obstacles)
    
    joint_points = forward_kinematics(arm, origin, joint_angles)
#     draw_joint_points(joint_points)
    
    free_spheres = collect(query_spheres(joint_points, obstacles))
#     draw_free_spheres(free_spheres)
    
    joint_safe_angles = arm.joint_highs - arm.joint_lows
    for i in 1:(length(free_spheres) - 1)
        safe_sphere = shared_sphere(free_spheres[i][2], free_spheres[i + 1][2])
        for joint in 1:free_spheres[i][1]
            joint_safe_angles[joint] = min(joint_safe_angles[joint],
                abs(tan(safe_sphere.radius / 
                        sum(joint_points[joint] .- safe_sphere.center) .^ 2)))
        end
    end
#     println("joint_angles ", joint_angles)
#     println("joint_safe_angles ", joint_safe_angles)
    return joint_safe_angles
end

configuration_distance (generic function with 1 method)

In [162]:
function sample(bounds)
  r = rand(length(bounds.highs))
  return Vector((bounds.highs .- bounds.lows) .* r .+ bounds.lows)
end

sample (generic function with 1 method)

In [193]:
function patch_tree_step!(tree::PatchTree, arm::Arm, origin, bounds,
        obstacles::Vector; patch_size_min=π/64) where {N}
    x = sample(bounds)
    near_patch = nearest_patch(tree, x)
    x_new = project(tree, near_patch, x)
#     println("x_new: ", x_new)
    patch_size = configuration_distance(arm, origin, x_new, obstacles)
    if minimum(patch_size) >= patch_size_min
        push_patch!(tree, near_patch, x_new, patch_size)
    end
end

patch_tree_step! (generic function with 1 method)

In [164]:
function push_patch!(tree::PatchTree, parent, center, patch_size)
    child = 1 + length(tree.parent)
    push!(tree.centers, center)
    push!(tree.patch_sizes, patch_size)
    push!(tree.parent, parent)
    push!(tree.children, [])
    push!(tree.children[parent], child)
end

push_patch! (generic function with 1 method)

In [165]:
function get_path(tree, last_idx::Int)::Vector
  path = [tree.centers[last_idx]]
  while last_idx != 1
    last_idx = tree.parent[last_idx]
    pushfirst!(path, tree.centers[last_idx])
  end
  return path
end

get_path (generic function with 1 method)

In [244]:
function patch_tree_goal(arm_origin, arm::Arm, initial_angles, bounds, obstacles, goal; patch_size_min=π/8)
  tree = PatchTree()
  patch_size = configuration_distance(arm, arm_origin, initial_angles, obstacles)
  push_patch!(tree, 1, initial_angles, patch_size)
  for i in 1:100000
#     println("tree: ", tree)
    gidx = nearest_patch(tree, goal)
    dist = patch_distance(tree.centers[gidx], tree.patch_sizes[gidx], goal)
#     println("distance: ", dist)
    if dist <= 0.
      path = get_path(tree, gidx)
      push!(path, goal)
      return tree, path
    end
    patch_tree_step!(tree, arm, arm_origin, bounds, obstacles, patch_size_min=patch_size_min)
  end
end

patch_tree_goal (generic function with 1 method)

In [245]:
obstacles = [[.5, .6, .7], [3., 1., 2.], [2., 1., 2.], [0., 0., 1.], ]
result_tree, path = patch_tree_goal([0., 0., 0.], arm, [0., 0.], DistancePlan.Box([-π, -π], [π, π]), obstacles, [π/2, -π/2])

(PatchTree(Any[[0.0, 0.0], [2.2655965916721934, 0.0], [-0.598785022776811, 0.0], [-0.598785022776811, -2.5773207151272923], [-0.598785022776811, -0.016542125635876277], [-0.598785022776811, 4.444836816959226], [-0.598785022776811, -1.7114612193413463], [-0.598785022776811, -6.452318354891472], [-0.598785022776811, 3.168794191036582], [2.2655965916721934, 0.26710447120653735]  …  [-0.598785022776811, -2.5224466513551764], [-1.1264322430662266, 4.6922106773750585], [2.2655965916721934, 0.1388206127903703], [-0.598785022776811, -2.7565560838524035], [-0.598785022776811, -0.24352824351912583], [-1.1264322430662266, -0.2442634960729526], [-0.598785022776811, 3.0521837886492937], [-0.598785022776811, -0.1349990634821895], [2.460076910560894, 0.06559971096743633], [2.022566096651009, 0.4558893060332612]], Any[[0.07540401167629347, 0.43434296305502], [0.4866530937930166, 0.4866530937930885], [0.46205263308565886, 1.4234579666376954], [0.40096743385584943, 0.79402146190501], [0.4900240406385284

In [246]:
path

11-element Array{Array{Float64,1},1}:
 [0.0, 0.0]                               
 [2.2655965916721934, 0.0]                
 [2.2319536683690355, 0.0]                
 [2.2319536683690355, 0.362774206731295]  
 [2.022566096651009, 0.362774206731295]   
 [2.022566096651009, 0.4073702063633373]  
 [2.022566096651009, 0.6688921416346647]  
 [2.022566096651009, 0.69108691427301]    
 [2.022566096651009, 0.5053798607741358]  
 [2.022566096651009, 0.4558893060332612]  
 [1.5707963267948966, -1.5707963267948966]

In [247]:
delete!(vis)
for (frame, angles) in enumerate(path)
    joint_points = forward_kinematics(arm, [0., 0., 0.], angles)
    draw_joint_points(joint_points, prefix=string("arm-frame", frame))
end
draw_obstacles(obstacles)